In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/Appliance-Energy-Prediction")
import pandas as pd
from pca_fts.PcaWeightedMVFTS import PcaWeightedMVFTS
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
import math

In [2]:
def sample_first_prows(data, perc=0.75):
    return data.head(int(len(data)*(perc)))

In [4]:
filename = '/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/HomeC.csv'
data = pd.read_csv(filename)
data.pop('time')
data.pop('House overall [kW]')
data.pop('cloudCover')
data.pop('summary')
data.pop('icon')
data.dropna(inplace=True)
data.head()

/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,use [kW],gen [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
0,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
1,0.934333,0.003467,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
2,0.931817,0.003467,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
3,1.022050,0.003483,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
4,1.139400,0.003467,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0


In [4]:
# diff = data.loc[:,'lights':'Tdewpoint']
# diff = diff.diff()
# diff['Appliances'] = data['Appliances']
# data = diff.dropna()

In [8]:
pca_wmvfts = PcaWeightedMVFTS(n_components = 2,
                       endogen_variable = 'use [kW]',
                       n_part = 50)

In [10]:
result = {
     "window": [],
     "rmse": [],
     "mape": [],
     "smape": []
}

tam = len(data)
n_windows = 30
windows_length = math.floor(tam / n_windows)
for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
    if len(ttest) > 0:
        
        print('-' * 20)
        print(f'training window {(ct)}')
        model, pca_reduced_train = pca_wmvfts.run_train_model(ttrain)
        forecast, forecast_self, pca_reduced_test = pca_wmvfts.run_test_model(model,ttest)
        
        print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
        rmse = Measures.rmse(pca_reduced_test['use [kW]'],forecast)
        mape = Measures.mape(pca_reduced_test['use [kW]'],forecast)
        smape = Measures.smape(pca_reduced_test['use [kW]'],forecast)
        
        result["rmse"].append(rmse)
        result["mape"].append(mape)
        result["smape"].append(smape)
        result["window"].append(ct)
        
measures = pd.DataFrame(result)



--------------------
training window 0
[ 23:28:28] getting statistics
--------------------
training window 16797


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/models/multivariate/mvfts.py:165: RuntimeWarning: invalid value encountered in double_scalars
  ret.append(np.dot(mv,mp.T)/np.nansum(mv))


[ 23:28:45] getting statistics
--------------------
training window 33594
[ 23:29:04] getting statistics
--------------------
training window 50391
[ 23:29:23] getting statistics
--------------------
training window 67188
[ 23:29:46] getting statistics
--------------------
training window 83985
[ 23:30:09] getting statistics
--------------------
training window 100782
[ 23:30:32] getting statistics
--------------------
training window 117579
[ 23:30:54] getting statistics
--------------------
training window 134376
[ 23:31:15] getting statistics
--------------------
training window 151173


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/models/multivariate/mvfts.py:165: RuntimeWarning: invalid value encountered in double_scalars
  ret.append(np.dot(mv,mp.T)/np.nansum(mv))


[ 23:31:34] getting statistics
--------------------
training window 167970
[ 23:31:51] getting statistics
--------------------
training window 184767
[ 23:32:11] getting statistics
--------------------
training window 201564
[ 23:32:31] getting statistics
--------------------
training window 218361
[ 23:32:49] getting statistics
--------------------
training window 235158
[ 23:33:07] getting statistics
--------------------
training window 251955
[ 23:33:26] getting statistics
--------------------
training window 268752
[ 23:33:45] getting statistics
--------------------
training window 285549
[ 23:34:05] getting statistics
--------------------
training window 302346
[ 23:34:24] getting statistics
--------------------
training window 319143
[ 23:34:45] getting statistics
--------------------
training window 335940
[ 23:35:04] getting statistics
--------------------
training window 352737
[ 23:35:23] getting statistics
--------------------
training window 369534


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/models/multivariate/mvfts.py:165: RuntimeWarning: invalid value encountered in double_scalars
  ret.append(np.dot(mv,mp.T)/np.nansum(mv))


[ 23:35:41] getting statistics
--------------------
training window 386331
[ 23:36:00] getting statistics
--------------------
training window 403128


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/models/multivariate/mvfts.py:165: RuntimeWarning: invalid value encountered in double_scalars
  ret.append(np.dot(mv,mp.T)/np.nansum(mv))


[ 23:36:18] getting statistics
--------------------
training window 419925
[ 23:36:36] getting statistics
--------------------
training window 436722
[ 23:36:54] getting statistics
--------------------
training window 453519
[ 23:37:12] getting statistics
--------------------
training window 470316
[ 23:37:32] getting statistics
--------------------
training window 487113
[ 23:37:51] getting statistics


In [11]:
measures

,window,rmse,mape,smape
0,0,0.050797,2.510324,1.188513
1,16797,0.117639,6.114907,2.896780
2,33594,0.103472,10.296154,3.878586
3,50391,0.117750,3.803799,1.949102
4,67188,0.084780,5.032981,2.391398
5,83985,0.073692,7.897894,3.847937
6,100782,0.064357,3.029002,1.533734
7,117579,0.071380,3.218636,1.496849
8,134376,0.182168,4.258831,1.833009
9,151173,0.072665,15.251885,4.765470


In [12]:
print("Mean: ")
print(statistics.mean(measures['rmse']))
print(statistics.mean(measures['mape']))
print(statistics.mean(measures['smape']))

Mean: 
0.10091633433377657
12.588217861184589
3.1194938969797112


In [22]:
train = sample_first_prows(data,0.75)
test = data.iloc[max(train.index):]
y_test = data.iloc[max(train.index):]['use [kW]'].values

In [23]:
pca_wmvfts = PcaWeightedMVFTS(n_components = 2,
                       endogen_variable = 'use [kW]',
                       n_part = 50)

In [25]:
# model, pca_reduced_train = pca_wmvfts.run_train_model(train)
# forecast, forecast_self, pca_reduced_test = pca_wmvfts.run_test_model(model,test)

In [ ]:
#model.mvflrg

In [ ]:
# forecast_1 = model.predict(pca_reduced_test)

In [ ]:
# Measures.rmse(y_test,forecast)

In [ ]:
# Measures.rmse(forecast_1,y_test)

In [ ]:
# Measures.rmse(y_test,forecast_self)

In [ ]:
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
# ax.plot(y_test, label='Original')
# ax.plot(forecast, label='Forecast')
# handles, labels = ax.get_legend_handles_labels()
# lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
# plt.show()


In [18]:
# rmse, mape, u = Measures.get_point_statistics(
#     data=pca_reduced_test,
#     model=model
# )

In [19]:
#rmse

66.18

In [20]:
#mape

27.09

In [21]:
pca_reduced_test

,x,y,Appliances
0,-0.536451,0.379525,280
1,-0.527063,0.400491,280
2,-0.519460,0.384878,320
3,-0.516230,0.379096,360
4,-0.503028,0.404434,210
...,...,...,...
4930,1.191707,0.317761,100
4931,1.179128,0.312629,90
4932,1.175407,0.296580,270
4933,1.151549,0.282911,420
